In [ ]:
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader 
import numpy as np
from scipy import io
import matplotlib.pyplot as plt
import argparse
import os
import copy
import time


def inital_soln(xt_grid):
    x_grid = xt_grid[:,:-1]
    t = xt_grid[:,-1].reshape(-1,1)

    d = x_grid.shape[1]
    u = torch.cos((30/d)*x_grid.sum(axis = 1)).reshape(-1,1)

    return u.flatten()


def exact_soln(xt_grid):
    
    x_grid = xt_grid[:,:-1]
    t = xt_grid[:,-1].reshape(-1,1)

    d = x_grid.shape[1]
    u = torch.cos((30/d)*x_grid.sum(axis = 1)).reshape(-1,1)*torch.exp(-t)
    
    return u.flatten()


def f_function(xt_grid):
    
    x_grid = xt_grid[:,:-1]
    t = xt_grid[:,-1].reshape(-1,1)

    d = x_grid.shape[1]
    u = (900/d-1)*torch.cos((30/d)*x_grid.sum(axis = 1)).reshape(-1,1)*torch.exp(-t)
    
    return u.flatten()


def stacked_grid(*args):
    # Generate meshgrid dynamically
    grids = torch.meshgrid(*args, indexing='ij')
    # Flatten and stack the grids into a single tensor
    stacked = torch.hstack([g.flatten()[:, None] for g in grids])
    return stacked.float()


def cal_domain_grad(model, XTGrid, device):
    Loss = torch.nn.MSELoss(reduction='mean')

    XTGrid = XTGrid.requires_grad_(True).to(device)
    u = model.forward(XTGrid)
    
    # Compute first derivatives
    u_grad = torch.autograd.grad(outputs=u, 
                                 inputs=XTGrid, 
                                 grad_outputs=torch.ones_like(u).to(device), 
                                 create_graph=True, 
                                 allow_unused=True)[0]

    # Compute second derivatives for each spatial dimension
    u_laplacian = torch.zeros_like(u[:,0], device=device)
    for d in range(XTGrid.shape[1] - 1):  # Assuming last dim is time
        ux = u_grad[:, d]
        uxx = torch.autograd.grad(outputs=ux, 
                                  inputs=XTGrid, 
                                  grad_outputs=torch.ones_like(ux).to(device), 
                                  create_graph=True, 
                                  allow_unused=True)[0][:, d]
        u_laplacian += uxx
    f = f_function(XTGrid)
    # Time derivative
    ut = u_grad[:, -1]  # Assuming last dimension is time

    # PDE residual loss (generalized heat/diffusion equation)
    lossf = Loss(ut-u_laplacian, f.flatten())

    # Compute gradient of loss function w.r.t. inputs (for adaptivity or error estimation)
    grad = torch.autograd.grad(outputs=lossf, 
                               inputs=XTGrid, 
                               grad_outputs=torch.ones_like(lossf).to(device),
                               create_graph=True,
                               allow_unused=True)[0]
    
    return grad


class RADSampler():
    def __init__(self, Nf, device, k, c, dim = 1):    
        self.device = device
        self.k = k
        self.c = c
        self.Nf = Nf
        self.dense_Nf = Nf*1
        self.dim = dim
        
    def update(self, model):
        
        x_new = torch.zeros(self.dense_Nf, self.dim, dtype = torch.float32, device=self.device).uniform_(-1, 1)
        t_new = torch.zeros(self.dense_Nf, 1, dtype = torch.float32, device=self.device).uniform_(0, 1)
        x_t_new = torch.concatenate((x_new,t_new), axis = 1)
        
        XTGrid = torch.tensor(x_t_new, dtype = torch.float32, device=self.device, requires_grad=True) 
        XTGrid = XTGrid.to(self.device)

        XTGrid = XTGrid.requires_grad_(True).to(self.device)
        
        u = model.forward(XTGrid)
        
        u_grad = torch.autograd.grad(outputs=u, 
                                     inputs=XTGrid, 
                                     grad_outputs=torch.ones_like(u).to(device), 
                                     create_graph=True, 
                                     allow_unused=True)[0]
    
        u_laplacian = torch.zeros_like(u[:,0], device=device)
        for d in range(XTGrid.shape[1] - 1):  # Assuming last dim is time
            ux = u_grad[:, d]
            uxx = torch.autograd.grad(outputs=ux, 
                                      inputs=XTGrid, 
                                      grad_outputs=torch.ones_like(ux).to(device), 
                                      create_graph=True, 
                                      allow_unused=True)[0][:, d]
            u_laplacian += uxx
            
        f = f_function(XTGrid)
        ut = u_grad[:, -1]  
    
        err = torch.abs((ut-u_laplacian-f.flatten()))
        err = (err**self.k)/((err**self.k).mean())+self.c
        err_norm = err/(err.sum())
        
        indice = torch.multinomial(err_norm, self.Nf, replacement = True)
        XTGrid = XTGrid[indice]
        self.XTGrid = XTGrid
        
class LASSampler():
    def __init__(self, Nf, fixed_uniform, device, L_iter = 1, beta = 0.2, tau = 0.002):
        self.Nf = Nf
        self.device = device
        self.cnt = 0
        self.beta = beta
        self.tau = tau
        self.L_iter = L_iter
        self.XTGrid = torch.tensor(copy.deepcopy(fixed_uniform), dtype=torch.float32, requires_grad=True).to(self.device)

    def update(self, phy_lf, model):

        x_data = self.XTGrid
        samples = x_data.clone().detach().requires_grad_(True)
        
        for t in range(1, self.L_iter + 1):
            grad = phy_lf(model, samples, self.device)
            scaler = torch.sqrt(torch.sum((grad+1e-16)**2, axis = 1)).reshape(-1,1)
            grad = grad/scaler

            with torch.no_grad():
                samples = samples + self.tau * grad + self.beta*torch.sqrt(torch.tensor(2 * self.tau, device=self.device)) * torch.randn(samples.shape, device=self.device)
                for i in range(samples.shape[1]):
                    if i < int(samples.shape[1]-1):
                        samples[:, i] = torch.clamp(samples[:, i], min=-1, max=1) 
                    else:
                        samples[:, i] = torch.clamp(samples[:, i], min=0, max=1)
                        
            samples = samples.clone().detach().requires_grad_(True)
        self.XTGrid = samples.detach()

class L_INFSampler():
    def __init__(self, Nf, device, step_size = 0.05 , n_iter = 20, dim = 1):

        self.Nf = Nf
        self.device = device
        self.step_size = step_size
        self.n_iter = n_iter
        self.dim = dim

    def update(self, phy_lf, model):

        x_new = torch.zeros(self.Nf, self.dim, dtype = torch.float32, device=self.device).uniform_(-1, 1)
        t_new = torch.zeros(self.Nf, 1, dtype = torch.float32, device=self.device).uniform_(0, 1)
        x_t_new = torch.concatenate((x_new,t_new), axis = 1)
        self.XTGrid = x_t_new
            
        x_data = self.XTGrid
        samples = x_data.clone().detach().requires_grad_(True)
    
        for t in range(1, self.n_iter + 1):
            grad = phy_lf(model, samples, self.device)
            with torch.no_grad():
                samples = samples + self.step_size * torch.sign(grad)
                for i in range(samples.shape[1]):
                    if i < int(samples.shape[1]-1):
                        samples[:, i] = torch.clamp(samples[:, i], min=-1, max=1)
                    else:
                        samples[:, i] = torch.clamp(samples[:, i], min=0, max=1)
                        
            samples = samples.clone().detach().requires_grad_(True)
        self.XTGrid = samples.detach()           


class R3Sampler(nn.Module):
    def __init__(self,Nf, fixed_uniform, device):
        super(R3Sampler, self).__init__()
        self.Nf = Nf
        self.device = device
        self.XTGrid = torch.tensor(copy.deepcopy(fixed_uniform), dtype=torch.float32, requires_grad=True).to(self.device)
    
    def update(self, loss_aver, loss_ele):
        with torch.no_grad():
            cho_i = loss_ele > loss_aver
            cho_i = cho_i.to('cpu')
            self.XTGrid = self.XTGrid[cho_i].detach()
            need_n_sample = self.Nf-self.XTGrid.shape[0]
            x_new = torch.zeros(need_n_sample, self.XTGrid.shape[1]-1, dtype = torch.float32, device=self.device).uniform_(-1, 1)
            t_new = torch.zeros(need_n_sample, 1, dtype = torch.float32, device=self.device).uniform_(0, 1)
            x_t_new = torch.concatenate((x_new,t_new), axis = 1)
            self.XTGrid = torch.concatenate((self.XTGrid, x_t_new), axis = 0)
            self.XTGrid = torch.tensor(self.XTGrid, dtype = torch.float32, device=self.device, requires_grad=True)
    
class PINN(nn.Module):
    def __init__(self,k , c , t, exact_XYT, exact_u, LBs, UBs, Layers, N0, Nb, Nf, 
                 Activation = nn.Tanh(), 
                 model_name = "PINN.model", device = 'cpu',
                  display_freq = 100, samp = 'fixed' ):
        
        super(PINN, self).__init__()
        
        
        self.LBs = torch.tensor(LBs, dtype=torch.float32)
        self.UBs = torch.tensor(UBs, dtype=torch.float32)
        
        self.Layers = Layers
        self.in_dim  = Layers[0]
        self.out_dim = Layers[-1]
        self.Activation = Activation
        
        self.device = device
        
        x_init = torch.zeros(Nf, len(self.LBs) - 1, dtype = torch.float32, device=self.device).uniform_(0, UBs[0])
        t_init = torch.zeros(Nf, 1, dtype = torch.float32, device=self.device).uniform_(0, UBs[-1])
        x_t_init = torch.concatenate((x_init,t_init), axis = 1)
        self.fixed_uniform = torch.tensor(x_t_init, dtype = torch.float32, device=self.device, requires_grad=True)
        
        self.N0 = N0
        self.Nb = Nb
        self.Nf = Nf
        
        self.t = t
        self.exact_XYT = exact_XYT
        self.exact_u = exact_u.reshape(-1,1)
        
        self.XT0, self.u0  = self.InitialCondition(self.N0, self.LBs, self.UBs)
        self.boundary_set = self.BoundaryCondition(self.Nb, self.LBs, self.UBs)
        
        self.XT0 = self.XT0.to(device)
        self.u0 = self.u0.to(device) 
        
        self._nn = self.build_model()
        self._nn.to(self.device)
        self.Loss = torch.nn.MSELoss(reduction='mean')
        
        self.model_name = model_name
        self.display_freq = display_freq
        
        self.k = k
        self.c = c
        self.X_star = self.exact_XYT
        self.method = samp
        
        self.r3_sample = R3Sampler(self.Nf, self.fixed_uniform, device)
        self.las = LASSampler(self.Nf, fixed_uniform=self.fixed_uniform, device=self.device, L_iter = 1, beta = 0.2, tau=1e-2)
        self.l_inf = L_INFSampler(self.Nf, device=self.device, step_size = 0.05 , n_iter = 20, dim = len(self.LBs) - 1)
        self.rad = RADSampler(Nf = self.Nf, device=self.device, k = self.k, c=self.c, dim = len(self.LBs) - 1)
        
    
    def build_model(self):
        Seq = nn.Sequential()
        for ii in range(len(self.Layers)-1):
            this_module = nn.Linear(self.Layers[ii], self.Layers[ii+1])
            nn.init.xavier_normal_(this_module.weight)
            Seq.add_module("Linear" + str(ii), this_module)
            if not ii == len(self.Layers)-2:
                Seq.add_module("Activation" + str(ii), self.Activation)
        return Seq
        
    def forward(self, x):
        return self._nn.forward(x.to(self.device)).reshape(-1)

    def InitialCondition(self, N0, LB, UB):
        n_per_dim = int(np.round(N0**(1/len(LB[:-1]))))
#         n_per_dim = int(np.round(np.sqrt(N0)))
        nodes = []
        for i in range(len(LB)-1):
            X = torch.linspace(LB[i], UB[i], n_per_dim).float()
            nodes.append(X)
            
        t_in = torch.tensor([LB[-1]]).float()
        nodes.append(t_in)
        
        XT_in = stacked_grid(*nodes)
        u0 = inital_soln(XT_in)
        return XT_in, u0

    def ICLoss(self):
        uv0_pred = self.forward(self.XT0)
        loss = self.Loss(uv0_pred, self.u0.to(self.device))
        return loss 

    def BoundaryPoints(self, nb, boundary_index, LBs, UBs, boundary_type):
        """
        Generalized function to get boundary points in an N-dimensional space.
        - `boundary_index`: Index of the dimension where the boundary is applied.
        - `nb`: Number of boundary points.
        - `LBs`, `UBs`: Lower and upper bounds for each dimension.
        - `boundary_type`: 'lower' for the lower boundary, 'upper' for the upper boundary.
        """
        n_per_dim = int(np.round(nb ** (1 / (len(LBs) - 1))))  # Distribute points across remaining dimensions
        
        # Create grid ranges for each dimension
        grid_ranges = []
        for i in range(len(LBs)  - 1):  # Excluding time
            if i == boundary_index:
                if boundary_type == 'lower':
                    grid_ranges.append(torch.tensor([LBs[i]]).float().to(self.device))  # Fixed lower boundary position
                else:
                    grid_ranges.append(torch.tensor([UBs[i]]).float().to(self.device))  # Fixed upper boundary position
            else:
                grid_ranges.append(torch.linspace(LBs[i], UBs[i], n_per_dim).float().to(self.device))

        # Time dimension (last dimension)
        grid_ranges.append(torch.linspace(LBs[-1], UBs[-1], nb).float().to(self.device))

        return stacked_grid(*grid_ranges)

    
    def BoundaryCondition(self, Nb, LBs, UBs):
        """
        Generalized function to get all boundary conditions in an N-dimensional space.
        - `Nb`: Total number of boundary points.
        - `LBs`, `UBs`: Lower and upper bounds for each dimension.
        """
        nb = int(np.round(Nb / (2 * (len(LBs) - 1))))  # Divide points across boundary faces
        boundary_sets = []
        
        for d in range(len(LBs) - 1):  # Iterate over all spatial dimensions
            lower_boundary = self.BoundaryPoints(nb, d, LBs, UBs, 'lower')
            upper_boundary = self.BoundaryPoints(nb, d, LBs, UBs, 'upper')
            boundary_sets.append((lower_boundary, upper_boundary))

        return boundary_sets  # Returns pairs of boundaries per dimension

    def BCLoss(self):
        """
        Compute boundary condition loss given a model, exact solution, and loss function.
        """
        total_loss = 0
        for lower, upper in self.boundary_set:
            U_L, U_R = self.forward(lower), self.forward(upper)
            UL_exact, UR_exact = exact_soln(lower), exact_soln(upper)

            total_loss += self.Loss(U_L, UL_exact) + self.Loss(U_R, UR_exact)
        
        return total_loss
    
    def PhysicsLoss(self, XTGrid):
        XTGrid = XTGrid.requires_grad_(True).to(device)
        u = self.forward(XTGrid)

        # Compute first derivatives
        u_grad = torch.autograd.grad(outputs=u, 
                                     inputs=XTGrid, 
                                     grad_outputs=torch.ones_like(u).to(device), 
                                     create_graph=True, 
                                     allow_unused=True)[0]

        # Compute second derivatives for each spatial dimension
        u_laplacian = torch.zeros_like(u, device=device)
        for d in range(XTGrid.shape[1] - 1):  # Assuming last dim is time
            ux = u_grad[:, d]
            uxx = torch.autograd.grad(outputs=ux, 
                                      inputs=XTGrid, 
                                      grad_outputs=torch.ones_like(ux).to(device), 
                                      create_graph=True, 
                                      allow_unused=True)[0][:, d]
            u_laplacian += uxx
            
        f = f_function(XTGrid)

        # Time derivative
        ut = u_grad[:, -1]  # Assuming last dimension is time

        loss2 = (u_laplacian - ut+f.flatten())**2
        loss1 = loss2.mean()

        return loss1, loss2 

    def Train(self, n_iters, weights=(1.0,1.0,1.0)):
        params = list(self.parameters())
        optimizer = optim.Adam(params, lr=1e-3)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 5000, gamma=0.9, last_epoch=-1)
        min_loss = 999999.0
        Training_Losses = [-10]*n_iters
        Test_Losses = []
        rel_error = [-10]*(1+n_iters//10)
        
        for jj in range(n_iters):
            Total_ICLoss = torch.tensor(0.0, dtype = torch.float32, device=self.device, requires_grad = True)
            Total_BCLoss = torch.tensor(0.0, dtype = torch.float32, device=self.device, requires_grad = True)
            Total_PhysicsLoss = torch.tensor(0.0, dtype = torch.float32, device=self.device, requires_grad = True)
            
            Total_ICLoss = Total_ICLoss + self.ICLoss()
            Total_BCLoss = Total_BCLoss + self.BCLoss()
            
            if self.method =='r3':
                if jj == 0:
                    XTGrid = self.r3_sample.XTGrid
                    XTGrid = torch.tensor(XTGrid, dtype = torch.float32, device=self.device, requires_grad=True) 
                else:
                    with torch.no_grad():
                        self.r3_sample.update(loss1, loss2)
                        XTGrid = self.r3_sample.XTGrid
                        XTGrid = torch.tensor(XTGrid, dtype = torch.float32, device=self.device, requires_grad=True) 
                        
            elif self.method == 'las':
                if jj == 0:
                    XTGrid = self.las.XTGrid
                    XTGrid = torch.tensor(XTGrid, dtype=torch.float32, requires_grad=True).to(self.device)
                else:
                    if self.las.cnt % 1 == 0:# 4,6,8,10 cnt = 4, 
                        self.las.update(cal_domain_grad, self._nn)
                    XTGrid = self.las.XTGrid
                    XTGrid = torch.tensor(XTGrid, dtype=torch.float32, requires_grad=True).to(self.device)
                self.las.cnt += 1
            
            elif self.method =='l_inf':
                    self.l_inf.update(cal_domain_grad, self._nn)
                    XTGrid = self.l_inf.XTGrid
                    XTGrid = torch.tensor(XTGrid, dtype=torch.float32, requires_grad=True).to(self.device)
                
            elif self.method =='rad':
                    self.rad.update(self._nn)
                    XTGrid = self.rad.XTGrid
                    XTGrid = torch.tensor(XTGrid, dtype=torch.float32, requires_grad=True).to(self.device)   
            
            elif self.method =='fixed':
                    XTGrid = torch.tensor(self.fixed_uniform, dtype = torch.float32, device=self.device, requires_grad=True) 
            
            elif self.method =='random-r':
                    x_new = torch.zeros(self.Nf, len(self.LBs) - 1, dtype = torch.float32, device=self.device).uniform_(-1, 1)
                    t_new = torch.zeros(self.Nf, 1, dtype = torch.float32, device=self.device).uniform_(0, 1)
                    x_t_new = torch.concatenate((x_new,t_new), axis = 1)
                    XTGrid = torch.tensor(x_t_new, dtype = torch.float32, device=self.device, requires_grad=True) 
                
            optimizer.zero_grad()    
            loss1, loss2 = self.PhysicsLoss(XTGrid) # For r3 method, loss2 contains element-wise errors
            
            Total_PhysicsLoss = Total_PhysicsLoss + loss1
            Total_Loss = weights[0]*Total_ICLoss + weights[1]*Total_BCLoss\
                        + weights[2]*Total_PhysicsLoss 
            
            Total_Loss.backward()
            optimizer.step()
            scheduler.step()
            if Total_Loss < min_loss:
                torch.save(self._nn.state_dict(), "../models/"+self.method+'_'+str(len(self.Layers)-2)+'_'+str(self.Nf)+'.pt')
                min_loss = float(Total_Loss)
                    
            Training_Losses[jj] = float(Total_Loss)
            
            if (jj+1) % self.display_freq == 0:
                with torch.no_grad():
                    outputs = self.forward(self.X_star)
                    outputs = outputs.reshape(-1,1)
                    re = np.linalg.norm(self.exact_u.cpu()-outputs.cpu().detach()) / np.linalg.norm(self.exact_u.cpu().detach())
                    rel_error[int((jj+1)/10)] = float(re*100)
                    
                print("Iteration Number = {}".format(jj+1))
                print("\tIC Loss = {}".format(float(Total_ICLoss)))
                print("\tBC Loss = {}".format(float(Total_BCLoss)))
                print("\tPhysics Loss = {}".format(float(Total_PhysicsLoss)))
                print("\tTraining Loss = {}".format(float(Total_Loss)))
                print("\tRelative L2 error (test) = {}".format(float(re*100)))

        return Training_Losses, rel_error


if __name__ == "__main__":
                
        parser = argparse.ArgumentParser()
        parser.add_argument('--nodes', type=int, default = 128, help='The number of nodes per hidden layer in the neural network')
        parser.add_argument('--layers', type=int, default = 8, help='The number of hidden layers in the neural network')
        parser.add_argument('--N0', type=int, default = 250, help='The number of points to use on the initial condition')
        parser.add_argument('--Nb', type=int, default = 250, help='The number of points to use on the boundary condition')
        parser.add_argument('--Nf', type=int, default = 1000, help='The number of collocation points to use')
        parser.add_argument('--epochs', type=int, default = 10000, help='The number of epochs to train the neural network')
        parser.add_argument('--method', type=str, default='random-r', help='Sampling method') # fixed, random-r, rad, r3, l_inf, las 
        parser.add_argument('--model-name', type=str, default='PINN_model', help='File name to save the model')
        parser.add_argument('--display-freq', type=int, default=10, help='How often to display loss information')
        parser.add_argument('-f')
        args = parser.parse_args()

        if not os.path.exists("../models/"):
            os.mkdir("../models/")
            
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        NHiddenLayers = args.layers

        desired_dim = 6
    
        LBs = [-1]*desired_dim
        UBs = [1]*desired_dim

        LBs.append(0)
        UBs.append(1)

        Nx = 8
        Nt = 8
        mesh_nodes = []
    
        for i in range(len(LBs)-1):
            X = torch.linspace(LBs[i], UBs[i], Nx).float()
            mesh_nodes.append(X)
        
        T = torch.linspace(LBs[-1], UBs[-1], Nt).float()
        mesh_nodes.append(T)
        
        XYT = stacked_grid(*mesh_nodes)
        Exact_U = exact_soln(XYT)
        Activation = nn.Tanh()
    
        Layers = [desired_dim+1] + [args.nodes]*NHiddenLayers + [1]

        k = 1
        c = 1

        repeat = [0, 1, 2, 3, 4]
        for i in repeat:
            pinn = PINN(  k = k,
                          c = c,
                          t= T,
                          exact_XYT = XYT,
                          exact_u = Exact_U,
                          Layers = Layers,
                          LBs = LBs,
                          UBs = UBs,
                          N0 = args.N0,
                          Nb = args.Nb,
                          Nf = args.Nf,
                          Activation = Activation,
                          device = device,
                          model_name = "../models/" + args.model_name + ".model_"+args.method+'_'+str(args.layers)+'_'+str(args.Nf)+'_'+str(i),
                          display_freq = args.display_freq, samp = args.method )

            Losses_train, Losses_rel_l2 = pinn.Train(args.epochs, weights = (1, 1, 1)) # initial, boundary, residual

            torch.save(Losses_train, "../models/" + args.model_name + ".loss_"+args.method+'_'+str(args.layers)+'_'+str(args.Nf)+'_'+str(i))
            torch.save(Losses_rel_l2, "../models/" + args.model_name + ".rel_l2_"+args.method+'_'+str(args.layers)+'_'+str(args.Nf)+'_'+str(i))

/tmp/ipykernel_1103819/2562604190.py:248: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.fixed_uniform = torch.tensor(x_t_init, dtype = torch.float32, device=self.device, requires_grad=True)
/tmp/ipykernel_1103819/2562604190.py:212: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.XTGrid = torch.tensor(copy.deepcopy(fixed_uniform), dtype=torch.float32, requires_grad=True).to(self.device)
/tmp/ipykernel_1103819/2562604190.py:151: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.XTGrid = torch.tensor(copy.deepcopy(fixed_uniform), dt

Iteration Number = 10
	IC Loss = 0.6168827414512634
	BC Loss = 5.069838523864746
	Physics Loss = 4752.1162109375
	Training Loss = 4757.802734375
	Relative L2 error (test) = 111.03110313415527
Iteration Number = 20
	IC Loss = 0.4871157109737396
	BC Loss = 3.6301751136779785
	Physics Loss = 5045.39501953125
	Training Loss = 5049.51220703125
	Relative L2 error (test) = 103.77471446990967
Iteration Number = 30
	IC Loss = 0.5014821887016296
	BC Loss = 3.6682448387145996
	Physics Loss = 5011.0703125
	Training Loss = 5015.240234375
	Relative L2 error (test) = 102.54703760147095
Iteration Number = 40
	IC Loss = 0.5001886487007141
	BC Loss = 3.6517395973205566
	Physics Loss = 4664.3232421875
	Training Loss = 4668.47509765625
	Relative L2 error (test) = 101.66395902633667
Iteration Number = 50
	IC Loss = 0.5001784563064575
	BC Loss = 3.637507915496826
	Physics Loss = 4942.18212890625
	Training Loss = 4946.31982421875
	Relative L2 error (test) = 100.68317651748657
Iteration Number = 60
	IC Loss =


KeyboardInterrupt

